In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Загрузка исходных данных в формате tsv в пандас датафрейм
column_names = [
    'account_id', 'name', 'point', 'call_count', 'total_call_time',
    'total_exclusive_time', 'min_call_time', 'max_call_time', 'sum_of_squares',
    'instances', 'language', 'app_name', 'app_id', 'scope', 'host',
    'display_host', 'pid', 'agent_version', 'labels'
]

df = pd.read_csv('metrics_collector.tsv', sep='\t', header=None, names=column_names, skiprows=1)
df.head()

,account_id,name,point,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,instances,language,app_name,app_id,scope,host,display_host,pid,agent_version,labels
0,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:33:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
1,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:34:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
2,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:35:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
3,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:36:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}
4,1,AgentCheck/CircuitBreaker/tripped/all,2024-04-15 23:37:00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,java,[GMonit] Collector,17592186045423,NaN,575f5ba20b4b,575f5ba20b4b,1,8.5.0,{}


# 1. Exploratory Analysis

In [3]:
def print_dataframe_shape(df):
    rows, columns = df.shape

    print(f"{'='*30}")
    print(f"Количество строк: {rows:,}".replace(',', ' '))
    print(f"Количество столбцов: {columns}")
    print(f"{'='*30}")

    
print_dataframe_shape(df)  

Количество строк: 18 363 749
Количество столбцов: 19


Из описания набора данных и ключевых метрик видно, что для всех четырех метрик анализа на первом этапе ('Web Response', 'Throughput', 'APDEX', 'Error') используются следующие общие условия фильтрации:

1. `language = 'java'`
2. `app_name = '[GMonit] Collector'` 
3. `scope = ''` – поле scope не заполнено \(пустое\).

Эти условия применяются ко всем запросам, что означает, что анализ ограничивается данными по Java-языку с именем приложения '[GMonit] Collector' и где поле scope не определено.

Также стоит отметить, что для метрики 'Error' используется дополнительное условие `name in ('HttpDispatcher', 'Errors/allWeb')`, однако это условие специфично только для этой метрики и не является общим для всех четырех метрик.

Выводы: общее условие для всех четырех метрик анализа на первом этапе одинаково и включает три критерия: \`language = 'java'\`, \`app_name = '\[GMonit\] Collector'\`, и \`scope = ''\` \(поле scope пустое\). Поэтому мы отфильтруем исходный датафрейм с учетом этих условий для дальнейшего анализа.

In [4]:
filtered_df = df[
                    (df['language'] == 'java') &
                    (df['app_name'] == '[GMonit] Collector') &
                    (df['scope'].isna()) &
                    (df['name'].isin(['Apdex', 'HttpDispatcher', 'Errors/allWeb']))
                ]

print_dataframe_shape(filtered_df)

Количество строк: 115 311
Количество столбцов: 19


In [5]:
filtered_df['point'] = pd.to_datetime(filtered_df['point'])

In [6]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115311 entries, 9772 to 18340409
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   account_id            115311 non-null  int64         
 1   name                  115311 non-null  object        
 2   point                 115311 non-null  datetime64[ns]
 3   call_count            115311 non-null  float64       
 4   total_call_time       115311 non-null  float64       
 5   total_exclusive_time  115311 non-null  float64       
 6   min_call_time         115311 non-null  float64       
 7   max_call_time         115311 non-null  float64       
 8   sum_of_squares        115311 non-null  float64       
 9   instances             115311 non-null  float64       
 10  language              115311 non-null  object        
 11  app_name              115311 non-null  object        
 12  app_id                115311 non-null  int64         

In [7]:
def get_categorical_columns(df):
    categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
    return categorical_columns

def unique_values_by_categorical_column(df):
    categorical_columns = get_categorical_columns(df)
    unique_values = {}
    for column in categorical_columns:
        unique_values[column] = df[column].unique().tolist()
    return unique_values

# Получаем уникальные значения для категориальных столбцов
unique_values = unique_values_by_categorical_column(filtered_df)

# Выводим уникальные значения
for column, values in unique_values.items():
    print(f"Уникальные значения для {column}: {values}")

Уникальные значения для name: ['Apdex', 'Errors/allWeb', 'HttpDispatcher']
Уникальные значения для language: ['java']
Уникальные значения для app_name: ['[GMonit] Collector']
Уникальные значения для scope: [nan]
Уникальные значения для host: ['575f5ba20b4b', '7d30f074a8d0']
Уникальные значения для display_host: ['575f5ba20b4b', '7d30f074a8d0']
Уникальные значения для agent_version: ['8.5.0', '8.7.0']
Уникальные значения для labels: ['{}']


In [8]:
# Подсчет уникальных значений для каждого указанного поля
unique_account_id = filtered_df['account_id'].nunique()
unique_pid = filtered_df['pid'].nunique()
unique_agent_version = filtered_df['agent_version'].nunique()
unique_app_id = filtered_df['app_id'].nunique()

print(f"Уникальных значений в 'account_id': {unique_account_id}")
print(f"Уникальных значений в 'pid': {unique_pid}")
print(f"Уникальных значений в 'agent_version': {unique_agent_version}")
print(f"Уникальных значений в 'app_id': {unique_app_id}")

Уникальных значений в 'account_id': 1
Уникальных значений в 'pid': 1
Уникальных значений в 'agent_version': 2
Уникальных значений в 'app_id': 1


- **labels**: Удаляем, так как пустое поле и не несет информации.
- **host и agent_version**: Удаляем, потому что дублирует `host_display` и создает избыточность.
- **app_name**, **account_id**, **pid**, **language**, app_id: Удаляем, так как содержит только одно значение, не способствует анализу.
- **scope**: Удаляем, все значения `nan`

In [9]:
# Удаление ненужных столбцов из filtered_df
columns_to_drop = ['labels', 'host', 'agent_version', 'app_name', 'language', 'account_id', 'pid', 'app_id', 'scope']
filtered_df = filtered_df.drop(columns=columns_to_drop)
filtered_df

,name,point,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,instances,display_host
9772,Apdex,2024-04-15 23:32:00,3896.0,1.500000,0.500000,0.500000,0.500000,0.000000,1.0,575f5ba20b4b
9773,Apdex,2024-04-15 23:33:00,3916.5,0.000000,1.000000,0.500000,0.500000,0.000000,1.0,575f5ba20b4b
9774,Apdex,2024-04-15 23:34:00,3991.5,0.500000,1.000000,0.500000,0.500000,0.000000,1.0,575f5ba20b4b
9775,Apdex,2024-04-15 23:35:00,3990.5,0.500000,0.500000,0.500000,0.500000,0.000000,1.0,575f5ba20b4b
9776,Apdex,2024-04-15 23:36:00,3912.5,2.000000,1.000000,0.500000,0.500000,0.000000,1.0,575f5ba20b4b
...,...,...,...,...,...,...,...,...,...,...
18340405,HttpDispatcher,2024-05-16 00:43:00,2760.0,38.546276,38.546276,0.000107,41.768820,873.722761,1.0,7d30f074a8d0
18340406,HttpDispatcher,2024-05-16 00:44:00,2693.5,15.862230,15.862230,0.000114,0.626102,1.083454,1.0,7d30f074a8d0
18340407,HttpDispatcher,2024-05-16 00:45:00,2668.5,16.437874,16.437874,0.000101,0.880042,1.486268,1.0,7d30f074a8d0
18340408,HttpDispatcher,2024-05-16 00:46:00,2654.0,18.585358,18.585358,0.000101,3.263524,7.259089,1.0,7d30f074a8d0


In [10]:
import pandas as pd

# Задание типа данных object для указанных столбцов
columns_to_convert = ['display_host', 'instances', 'name']
filtered_df[columns_to_convert] = filtered_df[columns_to_convert].astype('object')

filtered_df = filtered_df.rename(columns={'point': 'ds'})
filtered_df.isna().sum()

name                    0
ds                      0
call_count              0
total_call_time         0
total_exclusive_time    0
min_call_time           0
max_call_time           0
sum_of_squares          0
instances               0
display_host            0
dtype: int64

In [11]:
filtered_df.describe().T

,count,mean,std,min,25%,50%,75%,max
call_count,115311.0,2717.628938,1882.933801,0.066667,1560.000000,3106.000000,4145.500000,1.070100e+04
total_call_time,115311.0,22.647888,56.835851,0.000000,0.000000,7.000000,30.648737,7.221800e+03
total_exclusive_time,115311.0,21.495999,58.041630,0.000000,0.000000,3.000000,30.227040,7.221800e+03
min_call_time,115311.0,0.187584,0.242044,0.000000,0.000077,0.000109,0.500000,5.000000e-01
max_call_time,115311.0,6.465585,26.752252,0.000000,0.183849,0.500000,2.027876,5.777109e+02
sum_of_squares,115311.0,635.132579,15966.630607,0.000000,0.000000,0.000000,7.870205,3.185092e+06


### Потенциальные выбросы в данных:

- `total_call_time` и `total_exclusive_time`
  - **Максимальные значения** значительно отличаются от 75%-го процентиля.
  - **Возможные аномалии**: Аномально долгие времена вызовов.

- `max_call_time`
  - **Максимальное значение** сильно превышает 75%-й процентиль.
  - **Возможные аномалии**: Необычно долгие отдельные вызовы.

- `sum_of_squares`
  - **Стандартное отклонение** и **максимум** находятся на очень высоком уровне.
  - **Возможные аномалии**: Аномальные значения в расчетах метрики.

In [12]:
record_counts = filtered_df.groupby(['ds', 'name']).size().reset_index(name='record_count')
sorted_records = record_counts.sort_values(by=['record_count' , 'name' ], ascending=[False, False])
sorted_records.head(26)

,ds,name,record_count
25980,2024-04-22 18:36:00,HttpDispatcher,2
44349,2024-04-27 12:37:00,HttpDispatcher,2
72572,2024-05-04 20:39:00,HttpDispatcher,2
100063,2024-05-12 01:23:00,HttpDispatcher,2
103876,2024-05-13 00:58:00,HttpDispatcher,2
105962,2024-05-13 14:25:00,HttpDispatcher,2
110120,2024-05-14 16:38:00,HttpDispatcher,2
114196,2024-05-15 18:14:00,HttpDispatcher,2
114690,2024-05-15 21:09:00,HttpDispatcher,2
115098,2024-05-15 23:42:00,HttpDispatcher,2


In [13]:
# Обнаружение Множественных Записей за Минуту по Группам Name

filtered_df = filtered_df.merge(record_counts, on=['ds', 'name'])
filtered_df['mult_records'] = filtered_df['record_count'].apply(lambda x: 1 if x > 1 else 0)

In [14]:
mult_rec_df = filtered_df[filtered_df.mult_records == 1]
mult_rec_df.head(5)

,name,ds,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,instances,display_host,record_count,mult_records
9772,Apdex,2024-04-22 18:36:00,1932.5,4.0,1.0,0.5,0.5,0.0,0.5,575f5ba20b4b,2,1
9773,Apdex,2024-04-22 18:36:00,2003.0,2.0,1.0,0.5,0.5,0.0,0.5,575f5ba20b4b,2,1
9774,Apdex,2024-05-12 01:23:00,1198.5,0.0,1.0,0.5,0.5,0.0,0.5,575f5ba20b4b,2,1
9775,Apdex,2024-05-12 01:23:00,1192.0,0.0,0.5,0.5,0.5,0.0,0.5,575f5ba20b4b,2,1
11190,Apdex,2024-05-13 00:58:00,1297.5,2.0,0.5,0.5,0.5,0.0,0.5,575f5ba20b4b,2,1


# Формирование метрик

* Web Response, 
* Throughput
* APDEX
* Error

In [15]:
for column in ['name', 'instances', 'display_host']:
    unique_count = mult_rec_df[column].nunique()
    print(f'Уникальные значения в столбце {column}: {unique_count}')

Уникальные значения в столбце name: 3
Уникальные значения в столбце instances: 1
Уникальные значения в столбце display_host: 2


In [17]:
grouped_df = (
    filtered_df.groupby(['ds', 'name', 'instances', 'display_host', 'record_count', 'mult_records'], as_index=False)
    .agg({
        'call_count': 'sum',
        'total_call_time': 'sum',
        'total_exclusive_time': 'sum',
        'min_call_time': 'min',
        'max_call_time': 'max',
        'sum_of_squares': 'mean'
    })
)


grouped_df.head(5)

,ds,name,instances,display_host,record_count,mult_records,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares
0,2024-04-15 23:32:00,Apdex,1.0,575f5ba20b4b,1,0,3896.0,1.500000,0.500000,0.500000,0.500000,0.000000
1,2024-04-15 23:32:00,Errors/allWeb,0.5,575f5ba20b4b,1,0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000
2,2024-04-15 23:32:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,3898.0,30.099163,30.099163,0.000091,0.785708,2.049792
3,2024-04-15 23:33:00,Apdex,1.0,575f5ba20b4b,1,0,3916.5,0.000000,1.000000,0.500000,0.500000,0.000000
4,2024-04-15 23:33:00,Errors/allWeb,0.5,575f5ba20b4b,1,0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
record_counts = grouped_df.groupby(['ds', 'name']).size().reset_index(name='record_count')
# Сортировка результатов по 'ds' и 'name' в убывающем порядке
sorted_records = record_counts.sort_values(by=['record_count' , 'name' ], ascending=[False, False])
sorted_records

,ds,name,record_count
2,2024-04-15 23:32:00,HttpDispatcher,1
5,2024-04-15 23:33:00,HttpDispatcher,1
7,2024-04-15 23:34:00,HttpDispatcher,1
9,2024-04-15 23:35:00,HttpDispatcher,1
11,2024-04-15 23:36:00,HttpDispatcher,1
...,...,...,...
115274,2024-05-16 00:54:00,Apdex,1
115276,2024-05-16 00:55:00,Apdex,1
115278,2024-05-16 00:56:00,Apdex,1
115281,2024-05-16 00:57:00,Apdex,1


In [19]:
sorted_records.record_count.max()

1

## Первый набор данных: Web Response, Throughput 


- Web Response (Время ответа веб-сервиса):
Эта метрика измеряет время, которое требуется веб-сервису для ответа на внешний HTTP-запрос. Она важна для оценки производительности и быстродействия сервиса. Чем меньше время ответа, тем лучше.


- Throughput (Пропускная способность):
Эта метрика измеряет количество запросов, которые веб-сервис может обработать за определенный период времени (обычно в минуту). Пропускная способность важна для оценки способности сервиса обрабатывать высокий объем трафика. Чем выше пропускная способность, тем лучше.


Эти метрики вместе дают представление о том, насколько эффективно и быстро работает веб-сервис, что важно для обеспечения хорошего пользовательского опыта и стабильной работы системы.

In [21]:

http_metrics_df = grouped_df[grouped_df.name == 'HttpDispatcher']

# Вычисление метрик
http_metrics_df['web_response_time'] = http_metrics_df['total_call_time'] / filtered_df['call_count']
http_metrics_df['throughput'] = http_metrics_df['call_count']
http_metrics_df.drop(['total_call_time', 'call_count'], axis=1, inplace=True)
http_metrics_df.head(5)

,ds,name,instances,display_host,record_count,mult_records,total_exclusive_time,min_call_time,max_call_time,sum_of_squares,web_response_time,throughput
2,2024-04-15 23:32:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,30.099163,0.000091,0.785708,2.049792,0.007541,3898.0
5,2024-04-15 23:33:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,32.763374,0.000103,3.822411,8.938635,0.008417,3917.5
7,2024-04-15 23:34:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,34.776276,0.000110,3.822411,16.068309,0.009546,3993.0
9,2024-04-15 23:35:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,33.072502,0.000110,3.659296,8.886156,0.009014,3991.5
11,2024-04-15 23:36:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,42.575750,0.000107,15.111876,135.348995,0.012279,3915.5


In [25]:
http_metrics_df.to_csv('http_metrics.csv', index=False)

## Второй набор данных: APDEX 

APDEX (Application Performance Index) переводится как "Индекс производительности приложения". Этот индекс является сводным показателем, который используется для оценки "здоровья" и производительности приложения с точки зрения пользователя. Чем ближе значение APDEX к 1, тем лучше производительность и удовлетворенность пользователей.

In [27]:
# Фильтрация данных по условиям
apdex_metrics_df = grouped_df[grouped_df['name'] == 'Apdex']

# Вычисление метрики APDEX
apdex_metrics_df['s'] = apdex_metrics_df['call_count']
apdex_metrics_df['t'] = apdex_metrics_df['total_call_time']
apdex_metrics_df['f'] = apdex_metrics_df['total_exclusive_time']
apdex_metrics_df['apdex'] = (apdex_metrics_df['s'] + apdex_metrics_df['t'] / 2) / (apdex_metrics_df['s'] + apdex_metrics_df['t'] + apdex_metrics_df['f'])
apdex_metrics_df.drop(['s', 't', 'f', 'call_count', 'total_call_time', 'total_exclusive_time'], axis=1, inplace=True)

apdex_metrics_df.head(5)

,ds,name,instances,display_host,record_count,mult_records,min_call_time,max_call_time,sum_of_squares,apdex
0,2024-04-15 23:32:00,Apdex,1.0,575f5ba20b4b,1,0,0.5,0.5,0.0,0.999679
3,2024-04-15 23:33:00,Apdex,1.0,575f5ba20b4b,1,0,0.5,0.5,0.0,0.999745
6,2024-04-15 23:34:00,Apdex,1.0,575f5ba20b4b,1,0,0.5,0.5,0.0,0.999687
8,2024-04-15 23:35:00,Apdex,1.0,575f5ba20b4b,1,0,0.5,0.5,0.0,0.999812
10,2024-04-15 23:36:00,Apdex,1.0,575f5ba20b4b,1,0,0.5,0.5,0.0,0.999489


In [29]:
apdex_metrics_df.to_csv('apdex_metrics.csv', index=False)

## Третий набор данных: Error  

Процент ошибок в обработанных запросах. Эта метрика показывает долю запросов, которые завершились с ошибкой, относительно общего числа запросов.

In [45]:
# Отбор записей для метрики:
error_metrics_df = grouped_df[grouped_df['name'].isin(['HttpDispatcher', 'Errors/allWeb'])]
error_metrics_df.head(5)

,ds,name,instances,display_host,record_count,mult_records,call_count,total_call_time,total_exclusive_time,min_call_time,max_call_time,sum_of_squares
1,2024-04-15 23:32:00,Errors/allWeb,0.5,575f5ba20b4b,1,0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000
2,2024-04-15 23:32:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,3898.0,30.099163,30.099163,0.000091,0.785708,2.049792
4,2024-04-15 23:33:00,Errors/allWeb,0.5,575f5ba20b4b,1,0,0.5,0.000000,0.000000,0.000000,0.000000,0.000000
5,2024-04-15 23:33:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,3917.5,32.763374,32.763374,0.000103,3.822411,8.938635
7,2024-04-15 23:34:00,HttpDispatcher,1.0,575f5ba20b4b,1,0,3993.0,34.776276,34.776276,0.000110,3.822411,16.068309


In [46]:
# Группировка данных по времени
grouped_error = error_metrics_df.groupby(['ds', 'name']).agg({'call_count': 'sum'}).unstack(fill_value=0)
grouped_error

call_count               
name                Errors/allWeb HttpDispatcher
ds                                              
2024-04-15 23:32:00           0.5         3898.0
2024-04-15 23:33:00           0.5         3917.5
2024-04-15 23:34:00           0.0         3993.0
2024-04-15 23:35:00           0.0         3991.5
2024-04-15 23:36:00           0.0         3915.5
...                           ...            ...
2024-05-16 00:54:00           0.0         2560.0
2024-05-16 00:55:00           0.0         2491.0
2024-05-16 00:56:00           0.5         2429.0
2024-05-16 00:57:00           0.5         2373.5
2024-05-16 00:58:00           0.0         1170.0

[43272 rows x 2 columns]

In [47]:
flat_df = grouped_error.reset_index()
flat_df.columns = ['ds', 'call_count_errors', 'call_count_dispatcher']
flat_df['Error'] =  flat_df['call_count_errors'] / flat_df['call_count_dispatcher']

error_df = flat_df[['ds', 'Error']]
error_df.to_csv('error_ratio.csv')

,ds,call_count_errors,call_count_dispatcher,Error
0,2024-04-15 23:32:00,0.5,3898.0,0.000128
1,2024-04-15 23:33:00,0.5,3917.5,0.000128
2,2024-04-15 23:34:00,0.0,3993.0,0.000000
3,2024-04-15 23:35:00,0.0,3991.5,0.000000
4,2024-04-15 23:36:00,0.0,3915.5,0.000000
...,...,...,...,...
43267,2024-05-16 00:54:00,0.0,2560.0,0.000000
43268,2024-05-16 00:55:00,0.0,2491.0,0.000000
43269,2024-05-16 00:56:00,0.5,2429.0,0.000206
43270,2024-05-16 00:57:00,0.5,2373.5,0.000211


,ds,Error
0,2024-04-15 23:32:00,0.000128
1,2024-04-15 23:33:00,0.000128
2,2024-04-15 23:34:00,0.000000
3,2024-04-15 23:35:00,0.000000
4,2024-04-15 23:36:00,0.000000


,ds,Error
0,2024-04-15 23:32:00,0.000128
1,2024-04-15 23:33:00,0.000128
2,2024-04-15 23:34:00,0.000000
3,2024-04-15 23:35:00,0.000000
4,2024-04-15 23:36:00,0.000000
...,...,...
43267,2024-05-16 00:54:00,0.000000
43268,2024-05-16 00:55:00,0.000000
43269,2024-05-16 00:56:00,0.000206
43270,2024-05-16 00:57:00,0.000211
